The goal of this notebook is to create a student-teacher model where we first train a teacher on labeled data, and then use this teacher model to label more data, then we swap out the teacher with a student and train again over all the samples. 
- Try AutoAugment/RandAugment
- Add regularization
- Resampling  
  - Make resampling func for unlab_ds
- Create evaluation script
- Create new unlab_ds without previous findings

# Loading data

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import time
import os
import pathlib
import matplotlib.pyplot as plt
 
# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from data_prep_ttv import create_dataset, prepare_for_training
from utils import show_image, class_distribution
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

Some parameters

In [2]:
data_dir = pathlib.Path('/home/henrik/master-thesis/data/hyper-kvasir/labeled_split/')

conf = {
    # Dataset
    "data_dir": data_dir,
    "cache_dir": "./cache",
    "ds_info": 'hypkva',
    "augment": ["rotate","crop","flip","brightness","saturation","contrast"],
    "resample": True,
    "class_weight": False,
    "shuffle_buffer_size": 2000,        # 0=no shuffling
    "seed": 123,
    "neg_class": None,                 # select neg class for binary ds (normal class)
    "outcast": None,                   # list of folders to drop
    # Model
    "model": 'EfficientNetB0',
    "num_epochs": 40,
    "batch_size": 128,
    "img_shape": (128, 128, 3),
    "learning_rate": 0.01,
    "optimizer": 'Adam',
    "final_activation": 'softmax',
    # Callbacks
    "learning_schedule": True,
    "decay_rate": 0.15,                 # higher number gives steeper dropoff
    "checkpoint": False,
    "early_stopping": True,
    "early_stopping_patience": 5,
    # Misc
    "verbosity": 1
    }

model_name = '{}x{}x{}_{}_{}'.format(conf["num_epochs"], conf["batch_size"], 
                                     conf["img_shape"][1], conf["ds_info"], conf["model"])

Create training, testing and validation dataset from utils/data_prep.py.  
Returns tf.dataset for shuffled, cached and batched data

In [3]:
train_ds, test_ds, val_ds, params = create_dataset(conf)

train_steps = params["train_size"] // conf["batch_size"]
test_steps = params["test_size"] // conf["batch_size"]
val_steps = params["val_size"] // conf["batch_size"]
class_names = params["class_names"]

barretts-short-segment      :   53 | 0.50%
bbps-0-1                    :  646 | 6.06%
impacted-stool              :  131 | 1.23%
bbps-2-3                    : 1148 | 10.77%
hemorrhoids                 :    6 | 0.06%
ulcerative-colitis-grade-2  :  443 | 4.15%
normal-z-line               :  932 | 8.74%
retroflex-stomach           :  764 | 7.17%
esophagitis-b-d             :  260 | 2.44%
dyed-resection-margins      :  989 | 9.28%
ileum                       :    9 | 0.08%
ulcerative-colitis-0-1      :   35 | 0.33%
dyed-lifted-polyps          : 1002 | 9.40%
polyps                      : 1028 | 9.64%
ulcerative-colitis-2-3      :   28 | 0.26%
ulcerative-colitis-1-2      :   11 | 0.10%
ulcerative-colitis-grade-3  :  133 | 1.25%
retroflex-rectum            :  391 | 3.67%
esophagitis-a               :  403 | 3.78%
ulcerative-colitis-grade-1  :  201 | 1.89%
pylorus                     :  999 | 9.37%
cecum                       : 1009 | 9.46%
barretts                    :   41 | 0.38%

Total num

In [4]:
dist, _ = class_distribution(train_ds.unbatch().take(params["train_size"]), 23)
print (dist)

[0.00496378 0.06063858 0.01220821 0.10772739 0.00053662 0.04158841
 0.08746982 0.07163939 0.02441642 0.09283606 0.00080494 0.00321975
 0.09404346 0.09645828 0.00254897 0.00093909 0.01247652 0.03662463
 0.03783204 0.01878186 0.09377515 0.09471425 0.00375637]


In [ ]:
from utils import print_split_info

ds = {"train": train_ds.unbatch().take(params["train_size"]),
      "test": test_ds.unbatch().take(params["test_size"]),
      "val": val_ds.unbatch().take(params["val_size"])}

# NB: this will cache the whole dataset
print_split_info(ds, conf, params)

In [ ]:
import time
start_time = time.time()

# Show some images from training dataset (mainly to check augmentation)
batch = next(iter(train_ds))
images, labels = batch
images = images.numpy()

nrows, ncols = 3, 4  # array of sub-plots
figsize = [ncols*3, nrows*3]     # figure size, inches

# create figure (fig), and array of axes (ax)
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, 
                       figsize=figsize, frameon=False, facecolor='white')

# plot simple raster image on each sub-plot
for i, axi in enumerate(ax.flat):
    # i runs from 0 to (nrows*ncols-1)
    # axi is equivalent with ax[rowid][colid]
    img = images[i]
    axi.imshow(img)
    # get indices of row/column
    rowid = i // ncols
    colid = i % ncols
    # write row/col indices as axes' title for identification
    #axi.set_title("Row:"+str(rowid)+", Col:"+str(colid))
    axi.set_axis_off()

plt.axis('off')
plt.tight_layout(True)
# plt.savefig("{}/unlab_data_checkout-{}.pdf".format(log_dir, checkout), format='pdf')
plt.show()

print ("time:", time.time() - start_time)

# Step 1: Train a teacher model on labeled images

In [ ]:
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers

### Create and compile the model

In [ ]:
if conf["model"] == 'EfficientNetB0': 
    from efficientnet import EfficientNetB0 as EfficientNet # 5.3M params
elif conf["model"] == 'EfficientNetB1': 
    from efficientnet import EfficientNetB1 as EfficientNet # 7.8M params
elif conf["model"] == 'EfficientNetB2':
    from efficientnet import EfficientNetB2 as EfficientNet # 9.2M params
elif conf["model"] == 'EfficientNetB3':
    from efficientnet import EfficientNetB3 as EfficientNet # 12M params

In [ ]:
efficientnet_base = EfficientNet(
    weights="imagenet",
    include_top=False, 
    input_shape=conf["img_shape"]
)

# Unfreeze the layers. I.E we're just using the pre-trained weights as initial weigths and biases and train over them
efficientnet_base.trainable = True


# # Define model
# teacher_model = Sequential()
# teacher_model.add(efficientnet_base)
# teacher_model.add(layers.GlobalAveragePooling2D())
# teacher_model.add(layers.Dropout(0.3))
# teacher_model.add(layers.Dense(512, activation='relu'))
# teacher_model.add(layers.Dropout(0.3))
# teacher_model.add(layers.Dense(params["num_classes"], activation=conf["final_activation"]))

from keras import regularizers
# Define model
teacher_model = Sequential()
teacher_model.add(efficientnet_base)
teacher_model.add(layers.GlobalAveragePooling2D())
teacher_model.add(layers.Dropout(0.2))
teacher_model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001),
                               activation='relu'))
teacher_model.add(layers.Dropout(0.2))
teacher_model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001),
                               activation='relu'))
teacher_model.add(layers.Dropout(0.2))
teacher_model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001),
                               activation='relu'))
teacher_model.add(layers.Dropout(0.2))
teacher_model.add(layers.Dense(params["num_classes"], activation=conf["final_activation"]))

if conf['optimizer'] == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=conf["learning_rate"])
elif conf['optimizer'] == 'SGD':
    opt = tf.keras.optimizers.SGD(learning_rate=conf["learning_rate"])

teacher_model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

teacher_model.summary()

### Callbacks

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler
import datetime

In [ ]:
# By using LearnignRateScheduler
initial_learning_rate = conf["learning_rate"]
decay_steps = params["train_size"] // conf["batch_size"]
batch_size = conf['batch_size']
decay_rate = conf['decay_rate']

def schedule(epoch):
    # calculate new learning rate
    learning_rate = initial_learning_rate / (1 + decay_rate * (epoch*batch_size) / decay_steps)
    
    # update tensorboard
    tf.summary.scalar(name='learning_rate', data=learning_rate, step=epoch)
    return learning_rate

log_dir="./logs/{}/".format(conf["model"]) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()

lr_schedule_cb = LearningRateScheduler(schedule, verbose=1)
earlystopp_cb = EarlyStopping(monitor='val_loss', 
                              verbose=1, patience=conf["early_stopping_patience"], 
                              restore_best_weights=True)
checkpoint_cb = ModelCheckpoint(filepath='./models/best_cp-{epoch:03d}.hdf', 
                                monitor='val_loss', save_best_only=True, mode='auto')
tensorboard_cb = TensorBoard(log_dir=log_dir, update_freq='batch')

callbacks = [tensorboard_cb]
if conf["early_stopping"]: callbacks.append(earlystopp_cb)
if conf["learning_schedule"]: callbacks.append(lr_schedule_cb)
if conf["checkpoint"]: callbacks.append(checkpoint_cb)

# Write conf dictionary to text file
f = open(log_dir+"/conf.txt","w")
f.write(str(conf))
f.close()

# Write params dictionary to text file
f = open(log_dir+"/params.txt","w")
f.write(str(params))
f.close()

#### Class weight

In [ ]:
class_weights = None

if conf["class_weight"]:
    assert not conf["resample"], "Should only use resample or class_weight. Not both."
    
    from utils import calculate_weights

    class_weights = calculate_weights(
        train_ds.unbatch().take(params["train_size"]), 
        num_classes=params["num_classes"]
    )

    class_weights = dict(enumerate(class_weights))
    print ("Class weights:")
    print (class_weights)

### Train the teacher model

In [ ]:
start_time = time.time()
try: 
    history = teacher_model.fit(
        train_ds,
        steps_per_epoch = train_steps,
        epochs = conf["num_epochs"],
        validation_data = test_ds,
        validation_steps = test_steps,
        validation_freq = 1,
        class_weight = class_weights,
        callbacks = callbacks,
        verbose = 1
    )
except KeyboardInterrupt:
    print ("exiting")
    
finally:
    print ("Time spent on training: {}".format(time.time() - start_time))

In [ ]:
# Save the metrics from training
f = open(log_dir+"/history.txt","w")
f.write(str(history.history))
f.close()

### Save or restore a model

In [ ]:
# Save the model
best_acc =  str(history.history["val_sparse_categorical_accuracy"][-1])[2:4]
teacher_model.save(log_dir+'/{}.h5'.format(model_name+'_'+best_acc))

## Evaluate the model

In [ ]:
# Check if validation data is cached
cache_dir = conf["cache_dir"]
img_width = conf["img_shape"][0]
ds_info = conf["ds_info"]
filename = "{}/{}_{}_val.tfcache.index".format(cache_dir, img_width, ds_info)

cached = os.path.isfile(filename)
if not cached:
    # Iterate over dataset to initialize caching
    for batch in val_ds.take( (params["val_size"]//conf["batch_size"])+1 ):
        pass

In [ ]:
teacher_evaluate = teacher_model.evaluate(val_ds, verbose=2, steps=val_steps)

# Write evaluate dictionary to text file
f = open(log_dir+"/val_evaluate.txt","w")
f.write( str(teacher_evaluate) )
f.close()

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

if conf["learning_schedule"]: lr = history.history['lr']
epochs_range = range(history.epoch[-1]+1)

In [ ]:
if conf["learning_schedule"]:
    # Plot the learning rate
    plt.figure(figsize=(8, 6))
    plt.plot(epochs_range, lr, label='Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Learnign rate')
    plt.title('Learning Rate development during training');
    plt.savefig(log_dir+'/learning_rate.pdf', format='pdf')

In [ ]:
# Plot train-val accuracy and loss
plt.figure(figsize=(14, 6))

# Subplot 1
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
# plt.ylim([0.5, 1])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

# Subplot 2
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylim([0.0, 3])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Loss')
plt.savefig(log_dir+'/accuracy_and_loss.pdf', format='pdf')
plt.show()

### Test: run prediction on images from the validation dataset

### Test: run prediction on images from the unlabeled_test dataset
(which are images taken from the training data)  
TODO: just remove this?

# Step 2: use the teacher to generate pseudo labels on unlabeled images

### Read in the unlabeled dataset

In [ ]:
def get_filename(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    # the last item of parts is the filename
    filename = parts[-1]
    return filename

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img) # had to remove channels=3 paramter..
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [conf["img_shape"][0], conf["img_shape"][1] ])

def process_path(file_path):
    filename = get_filename(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, filename

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_dir_unlabeled = pathlib.Path('/home/henrik/master-thesis/data/hyper-kvasir/unlabeled_resized/')

ds_size_unlabeled = len(list(data_dir_unlabeled.glob('*.*g')))

files_string = str(data_dir_unlabeled/'*.*g')
list_ds_unlabeled = tf.data.Dataset.list_files(
        files_string, 
        shuffle=conf["shuffle_buffer_size"]>1, 
        seed=tf.constant(conf["seed"], tf.int64)
)

unlabeled_ds = list_ds_unlabeled.map(process_path, num_parallel_calls=AUTOTUNE)

print ("Loaded {} images into unlabeled_ds.".format(ds_size_unlabeled))

In [ ]:
# Run prediction on 1 image
#img, name = next(iter(unlabeled_ds))

for img, name in unlabeled_ds.take(1):

    print ("File:",str(name.numpy())[2:-1], end='\n\n')
    show_image(img.numpy())

    # Add dimension and predict
    img = np.expand_dims(img.numpy(), 0)
    prediction = teacher_model.predict(img, verbose=0)

    # If probability above 1% print info
    for i, pred in enumerate(prediction[0]):
         if pred > 0.01: print("{:>5.2f}% {}".format(pred*100, class_names[i]))

### Run predictions on all unlabeled images
Using 'append to list and convert to tensor'-method
- caching the dataset takes too much space

In [ ]:
from tqdm.notebook import tqdm

pred_confidence = 0.60
new_samples_counter = 0
count = 0
img_list = []
lab_list = []

In [ ]:
total_time = time.time()

tqdm_predicting = tqdm(total=ds_size_unlabeled, desc='Predicting', position=0, initial=count)
tqdm_findings = tqdm(total=ds_size_unlabeled, desc='Findings', 
                     position=1, bar_format='{desc}:{bar}{n_fmt}', initial=new_samples_counter)

print ("Press 'Interrupt Kernel' to save and exit.")
try:
    for count, (image,label) in enumerate(unlabeled_ds, start=count):
        img = np.expand_dims(image, 0)
        pred = teacher_model.predict(img)
        highest_pred = np.max(pred)
        if highest_pred > pred_confidence:
            pred_idx = np.argmax(pred).astype(np.int32)
            #pred_class = class_names[pred_idx]

            img_list.append(image)
            lab_list.append(pred_idx)
            # pred_list.append(highest_pred)
            
            new_samples_counter += 1
            tqdm_findings.update(1)
        tqdm_predicting.update(1)
except KeyboardInterrupt:
    print ("Exiting")

finally:
    print ("\nTotal run time: {:.3f} s".format( time.time() - total_time ))
    print ("Found {} new samples in unlabeled_ds after looking at {} images.".format(new_samples_counter, count))

Save the image and labels list as pickle dump

### Make bar chart of findings from unlabeled dataset

In [ ]:
lab_array = np.array(lab_list)
findings = np.bincount(lab_array, minlength=params["num_classes"])
assert len(class_names) == len(findings), "Must be same length."

# Settings
figsize=(15, 10)
# x = findings[:,0]
x = np.arange(params["num_classes"])
width = 0.5

fig, ax = plt.subplots(figsize=figsize)
# rects1 = ax.bar(x, findings[:,1], width, label='Findings')
rects1 = ax.bar(x, findings, width, label='Findings')
#rects2 = ax.bar(x + width/2, women_means, width, label='Women')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Number of samples')
ax.set_title("Found {} new samples in unlabeled_ds after looking at {} images.".format(new_samples_counter, count))
ax.set_xticks(x)
ax.set_xticklabels(class_names)
ax.set_axisbelow(True)
ax.legend()

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=25, ha="right",
         rotation_mode="anchor")
plt.grid(axis='y')

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)

fig.tight_layout()
plt.savefig(log_dir+'/unlab_data_prediction.pdf', format='pdf')
plt.show()

### Inspect the classified images

Create images with class label

In [ ]:
# Create images with label names
from PIL import Image, ImageDraw, ImageFont
import textwrap

class_label_img = []
font_path = '/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf'
# img_width = conf["img_shape"][0]
img_width = 512
font_size = int(img_width*0.15)
# print ('font size:',font_size)
letters_per_line = 13

for i in range(params["num_classes"]):
    img = Image.new('RGB', (img_width, img_width), color = (0, 0, 0))
    fnt = ImageFont.truetype(font_path, font_size)
    d = ImageDraw.Draw(img)
    if (len(class_names[i])>letters_per_line):
        text = textwrap.fill(class_names[i], width=letters_per_line)
    else:
        text = class_names[i]
    linebreaks = text.count('\n')
    d.text((1,(img_width//2.2)-linebreaks*img_width*0.1), text, font=fnt, fill=(255, 255, 255))
    
#     show_image(img)
    class_label_img.append(img)

Create a list containing 6 images from each predicted class

In [ ]:
# create a list[num_classes] with 6 sample images per class
# TODO: ADD SHUFFLING

# black image
img_black = Image.new('RGB', (conf["img_shape"][0], conf["img_shape"][1]), color = (0, 0, 0))

class_examples = []
for class_idx in range(params["num_classes"]):
    curr_list = []
    
    indekser = np.where(lab_array==class_idx)[0]
    for i in range(6):
        # get 6 finding images from class_idx-class
        # no image - index out of bounds
        if not indekser.size > i:
            curr_list.append(img_black)
        # found image
        else:
            curr_list.append(img_list[indekser[i]])
        
    class_examples.append(curr_list)

assert (len(class_names)==len(class_examples)), 'must be same length'

Display the predicted images in each class

In [ ]:
# settings
nrows, ncols = params["num_classes"], 7  # array of sub-plots
figsize = [ncols*2, params["num_classes"]*2]     # figure size, inches


# create figure (fig), and array of axes (ax)
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, 
                       figsize=figsize, frameon=False, facecolor='white')

# plot simple raster image on each sub-plot
try:
    for i, axi in enumerate(ax.flat):
        # i runs from 0 to (nrows*ncols-1)
        # axi is equivalent with ax[rowid][colid]
        rowid = i // ncols
        colid = i % ncols
        
        if colid == 0:
            img = class_label_img[rowid]
        else:
            img = class_examples[rowid][colid-1]
        axi.imshow(img)
        
        # write row/col indices as axes' title for identification
        #axi.set_title("Row:"+str(rowid)+", Col:"+str(colid))
        axi.set_axis_off()
except IndexError:
    pass

plt.axis('off')
plt.tight_layout(True)
plt.savefig("{}/unlab_data_checkout-{}.pdf".format(log_dir, 'all'), format='pdf')
plt.show()

In [ ]:
# The class we want to show images from
checkout = 'pylorus'
# checkout = 'polyps'
checkout = 'normal-z-line'

# Which class number correspond to that class name
idx = np.where(class_names == checkout)[0][0]
# List of img_list-indexes with images corresponding to that class number
idx_list = np.where(lab_list == idx)[0]

# This should be a function

# settings
nrows, ncols = 4, 6  # array of sub-plots
figsize = [15, 10]     # figure size, inches


# create figure (fig), and array of axes (ax)
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, 
                       figsize=figsize, frameon=False, facecolor='white')

# plot simple raster image on each sub-plot
for i, axi in enumerate(ax.flat):
    # i runs from 0 to (nrows*ncols-1)
    # axi is equivalent with ax[rowid][colid]
    img = img_list[idx_list[i]]
    axi.imshow(img)
    # get indices of row/column
    rowid = i // ncols
    colid = i % ncols
    # write row/col indices as axes' title for identification
    #axi.set_title("Row:"+str(rowid)+", Col:"+str(colid))
    axi.set_axis_off()

plt.axis('off')
plt.tight_layout(True)
plt.savefig("{}/unlab_data_checkout-{}.pdf".format(log_dir, checkout), format='pdf')
plt.show()

### Convert image and label lists to tensors and combine with training_ds to create a new dataset

In [ ]:
findings_tensor = tf.data.Dataset.from_tensor_slices((img_list, lab_list))

In [ ]:
# combine with old ds
combined_ds = train_ds.unbatch().take(params["train_size"]).concatenate(findings_tensor)

# Step 3: Train a student model on the combination of labeled images and pseudo labeled images

Now we have trained a teacher model, and used that model to predict on unlabeled dataset to create more samples with psudo-labels.  
It's time for swapping the teacher with the student!

In [ ]:
combined_ds = prepare_for_training(
        combined_ds, 
        conf["batch_size"], 
        cache=None,
        shuffle_buffer_size=0
    )

In [ ]:
# Unfreeze the layers. I.E we're just using the pre-trained weights as initial weigths and biases and train over them
efficientnet_base.trainable = True

# Define model
stud_model = Sequential()
stud_model.add(efficientnet_base)
stud_model.add(layers.GlobalAveragePooling2D())
# add Dense layer and dropout?
stud_model.add(layers.Dense(params["num_classes"], activation=conf["final_activation"]))

if conf['optimizer'] == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=conf["learning_rate"])
elif conf['optimizer'] == 'SGD':
    opt = tf.keras.optimizers.SGD(learning_rate=conf["learning_rate"])

stud_model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
train_steps = params["train_size"]+new_samples_counter // conf["batch_size"]

In [ ]:
history = stud_model.fit(
    combined_ds,
    steps_per_epoch = train_steps, # might have to reduce
    epochs = 2,
    validation_data = test_ds, # what to use for validation data?
    validation_steps = test_steps,
    validation_freq = 1,
    callbacks = callbacks
)

# Step 4: Iterate this algorithm a few times by treating the student as a teacher to relabel the unlabeled data and training a new student